In [ ]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [ ]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['model',
 'TGMT_360.jpg',
 'TGMT_40.jpg',
 '30.ipynb',
 '27.ipynb',
 '28.ipynb',
 '29.ipynb',
 'FaceImage',
 '31_data.ipynb',
 'done_data_face_features.pickle',
 '32_training_data.ipynb']

In [ ]:
# 1. DATA

In [ ]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/Test_opencv/done_data_face_features.pickle", mode="rb"))

In [ ]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.04440678  0.21475683  0.01395946 ... -0.0886332   0.06150215
   -0.00736863]]

 [[ 0.04842703  0.20101416  0.0238631  ... -0.08374722  0.07649732
    0.00379119]]

 [[ 0.04151651  0.19735098  0.02577441 ... -0.0647079   0.11292935
    0.02493233]]

 ...

 [[ 0.10115854  0.26688126 -0.05408768 ... -0.08945148  0.16420038
    0.0283456 ]]

 [[ 0.1297804   0.09853813 -0.07152011 ... -0.02761066  0.08419115
   -0.00754967]]

 [[ 0.06435601  0.00615203 -0.04063413 ...  0.11231432  0.06116884
    0.05675695]]]
['1721031561_NguyenDucHieu' '1721031561_NguyenDucHieu'
 '1721031561_NguyenDucHieu' ... '162000186_NguyenVanHuong'
 '162000186_NguyenVanHuong' '162000186_NguyenVanHuong']


In [ ]:
x.shape

(4700, 1, 128)

In [ ]:
y.shape

(4700,)

In [ ]:
x = x.reshape(-1, 128)

In [ ]:
x.shape

(4700, 128)

In [ ]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4230, 128), (470, 128), (4230,), (470,))

In [ ]:
# 2. Train machine learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [ ]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.73
F1 Score Train = 0.53
F1 Score Test = 0.54


In [ ]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.92
Accurency Test = 0.82
F1 Score Train = 0.82
F1 Score Test = 0.69


In [ ]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [ ]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.78
F1 Score Train = 1.00
F1 Score Test = 0.63


In [ ]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [ ]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.74


In [ ]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [ ]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  16.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  13.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.9s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

0.8546099290780141

In [ ]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/Test_opencv/ml_face_person_identity.pkl", mode='wb'))